In [566]:
import re
import pandas as pd
import numpy as np

In [567]:
class Node:
    """ Node class for a decision tree. """
    def __init__(self, names):
        self.names = names

    def classify(x):
        """ Handled by the subclasses. """
        return None

    def dump(self, indent):
        """ Handled by the subclasses. """
        return None


class Leaf(Node):
    def __init__(self, names, value):
        Node.__init__(self, names)
        self.value = value

    def classify(self, x):
        return self.value

    def dump(self, indent):
        print(' %d' % self.value)


class Split(Node):
    def __init__(self, names, var, left, right):
        Node.__init__(self, names)
        self.var = var
        self.left = left
        self.right = right

    def classify(self, x):
        if x[self.var] == 0:
            return self.left.classify(x)
        else:
            return self.right.classify(x)
      
    def dump(self, indent):
        if indent > 0:
            print('')
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 0 :' % self.names[self.var],end='')
        self.left.dump(indent+1)
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 1 :' % self.names[self.var],end='')
        self.right.dump(indent+1)


Helper function computes entropy of Bernoulli distribution with parameter p

In [568]:
def entropy(p):
    # >>>> YOUR CODE GOES HERE <<<<
    # For now, always return "0":
       
    entropy = (-p * np.log2(p) - (1 - p) * np.log2(1 - p)) if not (p == 0 or p == 1) else 0
    
    
    return entropy


In [569]:
#entropy(1/3)

In [570]:
#np.log2(0)

Compute information gain for a particular split, given the counts 

py_pxi : number of occurences of y=1 with x_i=1 for all i=1 to n

pxi : number of occurrences of x_i=1

py : number of ocurrences of y=1


In [571]:
def infogain(py_pxi, pxi, py, total):
    # >>>> YOUR CODE GOES HERE <<<<
    # For now, always return "0":
    
    #H1 = Positive Probability
    #H0 = Negative Probability
    
    '''
    Formula 
    Entropy(Dataset) – (Count(Group1) / Count(Dataset) * Entropy(Group1) + Count(Group2) / Count(Dataset) * Entropy(Group2))
    '''

    #Total Entropy of Sample
    H_s = entropy(py/total)
    
    # Positive probability for H=1
    H_1 = entropy(py_pxi/pxi) if (pxi != 0) else 0
        
       
    # Negative Probability for H=0
    H_0 = entropy((py - py_pxi)/(total - pxi)) if (pxi != total) else 0
    
    
    gain=0
    column_prob_group1= (pxi/total) 
    column_prob_group2= 1 - column_prob_group1

    
    gain = H_s - column_prob_group1  * H_1 - column_prob_group2 * H_0
    
    return gain


OTHER SUGGESTED HELPER FUNCTIONS:

-collect counts for each variable value with each class label

-find the best variable to split on, according to mutual information

-partition data based on a given variable	
	


In [572]:
# Load data from a file
def read_data(filename):
    f = open(filename, 'r')
    p = re.compile(',')
    data = []
    header = f.readline().strip()
    varnames = p.split(header)
    namehash = {}
    for l in f:
        data.append([int(x) for x in p.split(l.strip())])
    return (data, varnames)


Build tree in a top-down manner, selecting splits until we hit a pure leaf or all splits look bad.


In [573]:
def calculateCount(data,i):
    
    py_pxi = 0
    pxi = 0
    py = 0
    
    '''
    py_pxi : number of occurences of y=1 with x_i=1 for all i=1 to n

    pxi : number of occurrences of x_i=1

    py : number of ocurrences of y=1

    '''
    
    for row in data:
        
        if row[-1] == 1:      # For py : number of ocurrences of y=1
            py += 1
                
        
        
        if row[i] == 1:       #  For pxi : number of occurrences of x_i=1     
            pxi += 1
            
            
            if row[-1] == 1:  #  For py_pxi : number of occurences of y=1 with x_i=1 for all i=1 to n
                py_pxi += 1
                
                
    return(py_pxi,pxi,py)
    
    
    

def build_tree(data, varnames):
    # >>>> YOUR CODE GOES HERE <<<<
    # For now, always return a leaf predicting "1":
    
#     ## remove this part, it's here only as an example
#     #random tree
#     l1 = Leaf(varnames, 0)
#     r1 = Leaf(varnames, 1)
#     l0 = Leaf(varnames, 0)
#     r0 = Split(varnames, 2, l1, r1)
#     root = Split(varnames, 0 , r0, l1)
#     root.dump(0)
#     ####
    
    
    data_total = len(data)
    
    # Storing the data values based on the selected feature to left and right Values
    left_dataValues = []
    right_dataValues = []
    
    # To calculate py: number of ocurrences of y=1, that is to count the occurence on 1's in target value column
    py = 0
    for row in data:
        if row[-1] == 1:
            py += 1
            
    
    # Leaf Node Check
    if py == 0:
        return Leaf(varnames, 0)
    elif py == data_total:
        return Leaf(varnames, 1)

    
   
    gain,index_featureAttributes = float('-inf'),-1
    

    for i in range(len(varnames) - 1):
        if varnames[i]:
            
            py_pxi,pxi,py=calculateCount(data,i)

            # To Calculate Information Gain based on the current Feature Attribute
            feature_Gain = infogain(py_pxi, pxi, py, data_total)
            
            # Tracker To Store the Max Gain and corresponding feature
            if feature_Gain > gain:
                gain = feature_Gain
                index_featureAttributes = i
  
    
    for row in data:
        if row[index_featureAttributes] == 0:
            left_dataValues.append(row)
        else:
            right_dataValues.append(row)
    
    # Storing the Feature Attributes Name 
    left_features_Names = varnames[:]
    right_features_Names = varnames[:]
    
    #Creating Calculated checklist of Attributes
    left_features_Names[index_featureAttributes] = False
    right_features_Names[index_featureAttributes] = False
    
    # Recursively building the tree nodes
    leftNode = build_tree(left_dataValues, left_features_Names)
    rightNode = build_tree(right_dataValues, right_features_Names)
    
    
    # return the current node after building the subtrees
    
    root=Split(varnames, index_featureAttributes, leftNode, rightNode)
    
    return root


Here we load data.
Each example is a list of attribute values, where the last element in the list is the class value.


In [574]:
agaricus = ["agaricuslepiotatrain1.csv",
              "agaricuslepiotatest1.csv",
              "agaricuslepiotatest1.csv"]

dataset1 = ["data_sets1/training_set.csv",
            "data_sets1/validation_set.csv",
            "data_sets1/test_set.csv"]
            

dataset2 = ["data_sets2/training_set.csv",
            "data_sets2/validation_set.csv",
            "data_sets2/test_set.csv"]

# pick the dataset you want to use this time
# dataset = agaricus
# dataset = dataset1
# dataset = dataset2

dataset = agaricus

(train, varnames) = read_data(dataset[0])
(validation, validationvarnames) = read_data(dataset[1]) 
(test, testvarnames) = read_data(dataset[2]) 

In [575]:
pd.DataFrame(train).head()

,0,1,2,3,4,5,6,7,8,9,...,113,114,115,116,117,118,119,120,121,122
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [576]:
#varnames

Build the decision tree

In [577]:
root = build_tree(train, varnames)

In [578]:
root.dump(0)

odor-foul = 0 :
| gill-size-broad = 0 :
| | odor-none = 0 :
| | | gill-spacing-close = 0 :
| | | | bruises?-bruises = 0 : 1
| | | | bruises?-bruises = 1 : 0
| | | gill-spacing-close = 1 : 1
| | odor-none = 1 :
| | | stalk-surface-above-ring-silky = 0 :
| | | | bruises?-bruises = 0 : 0
| | | | bruises?-bruises = 1 : 1
| | | stalk-surface-above-ring-silky = 1 : 1
| gill-size-broad = 1 :
| | spore-print-color-green = 0 : 0
| | spore-print-color-green = 1 : 1
odor-foul = 1 : 1


Calcuating the accuracy

In [579]:
def accuracy(data):
    correct = 0
    # The position of the class label is the last element in the list.
    yi = len(data[0]) - 1
    for x in data:
    # Classification is done recursively by the node class.
    # This should work as-is.
        pred = root.classify(x)
        if pred == x[yi]:
            correct += 1
        acc = float(correct)/len(data)
    return acc;
   

In [580]:
 print("Train Accuracy: {}".format(accuracy(train)))

Train Accuracy: 1.0


In [581]:
 print("Test Accuracy: {}".format(accuracy(test)))

Test Accuracy: 0.9792941176470589
